In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

ERROR: Error in parse(text = x, srcfile = src): <text>:5:8: unexpected symbol
4: 
5: import numpy
          ^


****Company background ****

Bellabeat is a high-tech company founded in 2013 that manufactures health-focused smart products which inspires woman around the world. Bellabeat empower women with knowledge about their own health and habits by collecting data on activity, sleep, stress and reproductive health. 

The case study is done in order to reveal more opportunities for Bellabeat’s growth by getting insights on how people using their smart devices. By doing this, the company could figure out the trend and make adjustment on marketing strategies to promote growth. 


# **1.Ask Phase **


**Business Task **

To identify the trend of how consumers using smart devices and provide recommendations to the marketing team in order to apply the insight into latest marketing strategies. 

Who are the stakeholders: 

1)Urška Sršen: Bellabeat’s cofounder and Chief Creative Officer

2)Sando Mur: Mathematician and Bellabeat’s cofounder; key member of the Bellabeat executive team

3)Bellabeat marketing analytics team



# **2.Prepare Phase**

**2.1 Data used and information**

The data that we used in the analysis process was a public dataset, Fitbit Fitness Tracker Data(CC0: Public Domain, dataset made available through Mobius). This Kaggle data set contains personal fitness tracker from thirty fitbit users. Thirty eligible Fitbit users consented to the submission of personal tracker data, including minute-level output for physical activity, heart rate, and sleep monitoring. It includes information about daily activity, steps, and heart rate that can be used to explore users’ habit. 

We will use R in order to explore, analyze and visualize the data to find out the trend of how consumers use their smart devices. We also use SQL queries on the data frame by loading sqldf() .



**2.2 Install packages and import libraries**


In [ ]:
library(tidyverse)
library(janitor)
library(lubridate)
library(sqldf)
library(dplyr)
library(tidyr)
library(ggplot2)

**2.3 Import datas**

Data would be focus on – daily calories, daily steps , daily sleep.

In [ ]:
activity <- read_csv(file= "../input/fitbit/Fitabase Data 4.12.16-5.12.16/dailyActivity_merged.csv")
daily_sleep <- read_csv(file= "../input/fitbit/Fitabase Data 4.12.16-5.12.16/sleepDay_merged.csv")
hourly_steps <- read_csv("../input/fitbit/Fitabase Data 4.12.16-5.12.16/hourlySteps_merged.csv")
calories <- read.csv("../input/fitbit/Fitabase Data 4.12.16-5.12.16/hourlyCalories_merged.csv")


**Limitation **
As we explored the dataset, there are some limitations which we would need more dataset/ analysis in order to provide more solid recommendations before deciding the marketing strategy. 

1)	Data was last updated 2016- the statistics might not be the same currently as the latest data are not updated and during the past 3 years of pandemic, some of the consumers might have changed their lifestyle ( ex: work from home) thus more data are needed in order to keep the analysis up-to -date. 

2)	Small sample size – based on the data, there is only 33 distinct users which is small to represent the whole population size. 

3)	Participants demographic characteristics not provided- this might limit how well we could explore the data as we do not have any information about the gender as well as occupations, which even the end of case study might not be helpful for building the marketing strategies. 


**2.4 Preview data and columns**

We will preview selected data frames and summary of each. 

In [ ]:
head(activity)
head(daily_sleep)
head(hourly_steps)
head(calories)


# **3.Cleaning Data Process**

We will start process the data and clean some of the data to ease the process of analysis. The cleaning processes are dropping NA values, drop duplicates, naming some of the variables and standardize the format.

**3.1 Verify number of unique ID **


In [ ]:
n_distinct(activity$Id)
n_distinct(daily_sleep$Id)
n_distinct(hourly_steps$Id)
n_distinct(calories$Id)

After checking the count of user, then we will check if there is any duplicated rows in the tables. 

**3.2 Identify duplicated rows** 



In [ ]:
sum(duplicated(activity))
sum(duplicated(daily_sleep))
sum(duplicated(hourly_steps))
sum(duplicated(calories))

As we can see from the result, table " daily_sleep" contains 3 duplicated rows. We will proceed to remove the duplicated rows and check after cleaning it. 

**3.3 Remove Duplicates**


In [ ]:
daily_sleep <- daily_sleep %>% 
distinct() %>% 
drop_na()

We will verify the duplicated rows is being removed. 

In [ ]:
sum(duplicated(daily_sleep))

**3.4 Clean and rename the columns** 

We are going to rename the columns to standardize the column name and ensure the analysis process is more efficient without confusion. We will change all the column names to lower case. 

In [ ]:
clean_names(activity)
activity <- rename_with(activity, tolower)

clean_names(daily_sleep)
daily_sleep <- rename_with(daily_sleep, tolower)

clean_names(hourly_steps)
hourly_steps <- rename_with(hourly_steps, tolower)

clean_names(calories) 
calories <- rename_with(calories, tolower)


**3.5 Consistency of data and time columns before merging **

Before we merge the "activity" and "sleep" table, we would like to ensure the consistency in terms of format for date & time column. 


In [ ]:
activity <- activity %>% 
rename(date=activitydate)%>%
mutate(date=as_date(date,format = "%m/%d/%Y"))

daily_sleep <- daily_sleep %>%
rename(date = sleepday) %>%
mutate(date = as_date(date,format ="%m/%d/%Y %I:%M:%S %p" , tz=Sys.timezone()))

head(activity)
head(daily_sleep)

**3.5 Convert data_string to date_time ***

We will convert data string into date_time for both "hourly_step" and "calories".



In [ ]:
hourly_steps<- hourly_steps %>% 
rename(date_time = activityhour) %>% 
mutate(date_time = as.POSIXct(date_time,format ="%m/%d/%Y %I:%M:%S %p" , tz=Sys.timezone()))
  
calories<- calories %>% 
rename(date_time = activityhour) %>% 
mutate(date_time = as.POSIXct(date_time,format ="%m/%d/%Y %I:%M:%S %p" , tz=Sys.timezone()))

head(hourly_steps)
head(calories)

**3.7 Merging dataset **

We will merge both dataset ("daily_sleep" and "activity") in order to identify if there is any correlation between these variables. 



In [ ]:
dailyactivity_sleep<- merge(activity,daily_sleep, by=c("id","date"))
glimpse(dailyactivity_sleep)

3.8 Summary of all data 

We will take a look on the summaries of all the data. 

In [ ]:
summary(dailyactivity_sleep)




In [ ]:
summary(hourly_steps)
summary(calories)

# **4.Analyse Phase **

We will analyse the user trend of fitbits and identify what are the keys could be used as the marketing strategy for Bellabeat. 

These are some of the questions we could brainstorm in order to identify the trend: 
* How often the users wear smart devices? Group the users on usage frequency?
* What are the lifestyles of the users? Active or moderate or sedentary? ( since the demographic data are not provided, we could group the users by average steps) 
* Most active hours ? the calories graph – to identify the active hours of users 
* Correlation between steps/calories/ sleeptime? 


***4.1 Identify how often Fitbit users use their smart devices *******

We would like to identify how often users use their smart device to track their daily activities by classifying them into 3 categories: 
1. Uses device between 21-31 days - frequent 
2. Uses device between 11-20 days - on and off 
3. Uses device between 1-10 days - low 


In [ ]:
daily_use <- dailyactivity_sleep %>%
group_by(id) %>%
summarize(days_used=sum(n())) %>%
mutate(user_stage=case_when (
days_used >= 1 & days_used <= 10  ~ "low",
days_used>= 11 & days_used <= 20  ~ "on and off",
days_used>= 21 & days_used <= 31 ~ "frequent"))

head(daily_use)

In [ ]:
daily_count <- daily_use %>%
group_by(user_stage) %>% 
summarize(user=n())

head(daily_count)


Then we will summarize the daily_use data into percentage. 

In [ ]:
daily_use_percentage <- daily_use %>% 
group_by(user_stage) %>% 
summarize(total = n()) %>%
mutate(totals =sum(total))%>%
group_by(user_stage) %>%
summarize(total_percentage= total/totals*100)


head(daily_use_percentage)

In [ ]:
daily_use_percentage %>%
ggplot(aes(x=user_stage,y=total_percentage,fill=user_stage))+geom_bar(stat="identity")+labs(title="Usage of smart device")

We can conclude from the graph above that

50% of the users are using their smart devices in between 21-31 days which is 12 users in the data. 
37.5% of the users are using the devices in between 1-10 days which is 9 users in the data. 

More data are needed in order to determine any reason that cause the difference in terms of usage so that we could help to improve the usage rate of smart devices for our users as well.  


****4.2 Active hours of users based on calories and steps ******



In [ ]:
calories <- calories %>%
  separate(date_time, into = c("date", "time"), sep= " ") %>%
  mutate(date = ymd(date)) 

In [ ]:
calories %>%
group_by(time) %>% 
summarize(average_cal = mean(calories))%>%
ggplot()+
geom_col(mapping=aes(x=time,y=average_cal,fill=average_cal)) +
labs(title= "Hourly calories throughout the day",x="",y="")+
scale_fill_gradient(low="green",high="red")+
theme(axis.text.x= element_text(angle= 90))

In [ ]:
hourly_steps <- hourly_steps %>%
  separate(date_time, into = c("date", "time"), sep= " ") %>%
  mutate(date = ymd(date)) 

In [ ]:
head(hourly_steps)

In [ ]:
hourly_steps %>%
group_by(time) %>% 
summarize(average_steps = mean(steptotal))%>%
ggplot()+
geom_col(mapping=aes(x=time,y=average_steps,fill=average_steps)) +
labs(title= "Hourly steps throughout the day",x="",y="")+
scale_fill_gradient(low="green",high="red")+
theme(axis.text.x= element_text(angle= 90))

In the graph above, we can determine that: 

* the most active time for users is 12pm-2pm and 5pm-7pm. 
* For 12pm-2pm could be during lunchtime whereas for 5pm-7pm could be dinner time or workout after work.  

****4.3 Average sleep hours ****

We are going to analyse the average sleep hours per users in order to identify the trend since we do not have exact demographic details of users.  




In [ ]:
daily_sleep <- dailyactivity_sleep %>%
group_by(id) %>%
summarize(average_sleep_hour=mean(totalminutesasleep)/60)

head(daily_sleep)

In [ ]:
daily_sleep %>%
ggplot()+geom_point(mapping=aes(x=id,y=average_sleep_hour))+labs(title="Average sleep for users")

Findings: 

The average hours of sleep is between 6-8 hours for users in the data. 


****4.4 Correlations** **

We would find out is there is any correlation between these variables: 
* Daily steps vs calories 
* Daily steps vs daily sleep

In [ ]:
ggplot(dailyactivity_sleep,aes(x=totalsteps,y=totalminutesasleep ))+geom_jitter()+geom_smooth(color="blue")+labs(title="Daily steps vs total minutes asleep",x="Daily steps",y="Minute asleep")+theme(plot.title=element_text(size=20))

In [ ]:
ggplot(dailyactivity_sleep,aes(x=totalsteps,y=calories ))+geom_jitter()+geom_smooth(color="green")+labs(title="Daily steps vs calories burned",x="Daily steps",y="Calories burned")+theme(plot.title=element_text(size=20))

According our plots: 

* There is positive correlation between daily steps and calories as more steps burn more calories. 
* There is no correlation between daily steps and amount of sleep time. 

**4.5 User activeness based on calories **

We would like to identify the user activeness in this group of users by classifying them into 3 groups. 

* mean calories < 2000 - sedentary
* mean calories between 2000 and 2999 -active 
* mean calories >= 3000 - very active

In [ ]:
user_activeness <- dailyactivity_sleep %>% 
group_by(id)%>%
summarize(mean_calories = mean(calories),mean_steps=mean(totalsteps))

head(user_activeness)


In [ ]:
user_type <- user_activeness %>% 
mutate(user_type= case_when( mean_calories >= 3000 ~ "very active",
                            mean_calories >=2000  & mean_calories <2999 ~ "active",
                            mean_calories < 2000 ~ "sedentary" ))

head(user_type)
summary(user_type)

In [ ]:
user_type_percent <- user_type %>%
group_by(user_type) %>%
summarise(total=n()) %>%
mutate(totals=sum(total))%>%
group_by(user_type)%>%
summarize(total_percent= total/totals) %>%
mutate ( perc= scales::percent(total_percent))

user_type_percent$user_type <- factor(user_type_percent$user_type,levels =c("very active","active","sedentary"))

head(user_type_percent)

In [ ]:
user_type_percent %>%
ggplot(aes(x="",y=total_percent,fill=user_type))+
geom_bar(stat="identity",width=1)+
coord_polar("y",start=0)+
theme_minimal()+
theme(axis.title.x= element_blank(),
axis.title.y = element_blank(),
panel.border = element_blank(), 
panel.grid = element_blank(), 
axis.ticks = element_blank(),
axis.text.x = element_blank(),
plot.title = element_text(hjust = 0.5, size=14, face = "bold"))+
geom_text(aes(label=perc),position= position_stack(vjust =0.5))+labs(title="User activeness")



Based on the pie chart: 

We can observe that 66.7% of users are categorize between active (2000-2999 calories per day) and very active( equal or more than 3000 calories per day). 

More data would be needed in order to analyse further according to user activeness. 

# **5.Act Phase **

**Conclusion **

In order for us to help Bellabeat's team to improve their marketing strategies, I would recommend to collect more data in details such as demographic data since we have a specific target customers - young women adult. The current dataset used a small sample sizes which might not be as accurate or could be highly biased. 
Moreover, if we manage to collect the latest data it would be easier to identify the current usage trend and we could discuss further on the marketing strategies. 

Based on the findings, 

How often users use smart device 
Findings= 50% of the users are using their device in between 21-31 days. 
We could promote the long last batteries of our device as the big advantage of using our tracker. However more analysis needed to be done on our own tracking data so we could determine what is the usage rate among customers. 


User activeness based on calories= 
Findings= 67% of users in the dataset consumed at least 2000 calories per day. 
We could improve our device by sending notification to promote active lifestyle. If we could create any programme that allow user to calculate their own BMR rate and how much calories they should burn to be in active lifestyle, it would be really useful especially for users who are in active lifestyle or starting diet plan. 

That's the end of my first project. 

Thank you! 
